* Density + heatmap
* Region
* more stuff

In [58]:
%matplotlib inline

import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

import plotly
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
plotly.offline.init_notebook_mode()

import time
import collections as col
from collections import OrderedDict
import ast

from ndreg import *
import ndio.remote.neurodata as neurodata
import nibabel as nib
import networkx as nx
import re
import pandas as pd

import requests
import json

import seaborn as sns

import csv

In [13]:
def time_function(fun, *args):
    start = time.time()
    result = fun(*args)
    run_time = time.time() - start
    print('run time: %f s' % run_time)
    return result

# Density Graph stuff

In [20]:
## This finds the maximum number of edges and the densest node.
G = nx.read_graphml('Fear199_backup.graphml')

maxEdges = 0
densestNode = ""
#     for i in range(len(G.edges())):
#         if ((len(G.edges('s' + str(i))) > maxEdges)):
#             maxEdges = len(G.edges('s' + str(i)))
#             densestNode = "s" + str(i)

for node in G.nodes():
    num_edges = len(G.edges(node))
    if (num_edges > maxEdges):
        maxEdges = num_edges
        densestNode = node

## Find each node with a given number of edges, from 0 edges to maxEdges

## Find and store number of edges for each node in storageDict
## Key is 's1', value is number of edges
storageDict = {}

for n in G.nodes_iter():
    storageDict[n] = len(G.edges(n))

orderedNodesEdgeCounts = col.OrderedDict(sorted(storageDict.items(), key=lambda (key, value): int(key.split('s')[1])))


In [25]:
## Create ordered list to visualize data
sortedList = sorted(storageDict.values())

# Calculate basic statistics
statisticsArray = np.array(sortedList)
averageNumberEdges = np.mean(statisticsArray)
stdNumberEdges = np.std(statisticsArray)

print("average edge count:")
print(averageNumberEdges)
print("standard deviation edge count: ")
print(stdNumberEdges)

# using 95th percentile as upper limit (z = 1.96)
upperLimit = averageNumberEdges + 1.96 * stdNumberEdges
print("95th percentile: ")
print(upperLimit)

##unused

## numberEdges is used for plotting (first element is edges for 's1', etc.)
numberEdges = []
k = 0
for i in range(1, (len(G.nodes()) + 1)):
    numberEdges.append(orderedNodesEdgeCounts['s' + str(i)])
    k = k + 1

# for node in G.nodes():
#     numberEdges.append(orderedNodesEdgeCounts[node])

## Number of colors is maxEdges
numColors = maxEdges;

# scaledEdges = [float(numberEdges[i])/float(upperLimit) for i in range(len(numberEdges))]
scaledEdges = [float(numberEdges[i])/float(maxEdges) for i in range(len(numberEdges))]

average edge count:
11.7980613893
standard deviation edge count: 
4.18959644527
95th percentile: 
20.0096704221


In [26]:
print(numberEdges)

[5, 10, 1, 3, 11, 11, 11, 3, 9, 6, 7, 7, 7, 4, 5, 9, 7, 14, 10, 7, 8, 5, 14, 10, 7, 8, 5, 8, 8, 10, 16, 10, 16, 10, 9, 1, 2, 4, 8, 8, 6, 8, 4, 12, 12, 12, 10, 10, 9, 6, 7, 7, 10, 12, 16, 13, 9, 15, 13, 4, 3, 1, 5, 9, 14, 8, 8, 4, 12, 12, 11, 1, 6, 4, 16, 10, 9, 4, 5, 5, 16, 10, 15, 7, 5, 7, 7, 6, 7, 4, 7, 14, 16, 6, 9, 15, 7, 12, 8, 8, 14, 16, 15, 21, 12, 11, 16, 5, 11, 7, 12, 9, 13, 13, 13, 6, 5, 7, 5, 9, 5, 12, 7, 9, 7, 11, 10, 5, 13, 14, 7, 19, 8, 5, 8, 11, 12, 9, 5, 11, 14, 14, 14, 16, 13, 11, 11, 13, 6, 10, 14, 12, 11, 8, 1, 8, 12, 16, 8, 13, 11, 14, 13, 15, 13, 6, 13, 10, 3, 7, 8, 12, 11, 10, 13, 18, 20, 15, 16, 12, 10, 16, 9, 2, 16, 15, 12, 17, 20, 10, 11, 14, 13, 22, 12, 14, 5, 13, 12, 12, 12, 19, 19, 15, 19, 15, 7, 5, 7, 3, 14, 6, 14, 17, 7, 14, 14, 15, 15, 14, 7, 14, 11, 14, 6, 7, 16, 11, 11, 14, 16, 17, 13, 11, 11, 17, 16, 12, 13, 11, 16, 11, 9, 14, 4, 15, 10, 17, 15, 16, 12, 13, 14, 14, 12, 11, 12, 1, 3, 6, 12, 14, 19, 11, 15, 19, 17, 17, 14, 10, 17, 13, 12, 12, 7, 5, 4, 11

In [23]:
print(G.edges('s1'))

[('s1', 's35'), ('s1', 's44'), ('s1', 's71'), ('s1', 's70'), ('s1', 's47')]


In [21]:
print(orderedNodesEdgeCounts)

OrderedDict([('s1', 5), ('s2', 10), ('s3', 1), ('s4', 3), ('s5', 11), ('s6', 11), ('s7', 11), ('s8', 3), ('s9', 9), ('s10', 6), ('s11', 7), ('s12', 7), ('s13', 7), ('s14', 4), ('s15', 5), ('s16', 9), ('s17', 7), ('s18', 14), ('s19', 10), ('s20', 7), ('s21', 8), ('s22', 5), ('s23', 14), ('s24', 10), ('s25', 7), ('s26', 8), ('s27', 5), ('s28', 8), ('s29', 8), ('s30', 10), ('s31', 16), ('s32', 10), ('s33', 16), ('s34', 10), ('s35', 9), ('s36', 1), ('s37', 2), ('s38', 4), ('s39', 8), ('s40', 8), ('s41', 6), ('s42', 8), ('s43', 4), ('s44', 12), ('s45', 12), ('s46', 12), ('s47', 10), ('s48', 10), ('s49', 9), ('s50', 6), ('s51', 7), ('s52', 7), ('s53', 10), ('s54', 12), ('s55', 16), ('s56', 13), ('s57', 9), ('s58', 15), ('s59', 13), ('s60', 4), ('s61', 3), ('s62', 1), ('s63', 5), ('s64', 9), ('s65', 14), ('s66', 8), ('s67', 8), ('s68', 4), ('s69', 12), ('s70', 12), ('s71', 11), ('s72', 1), ('s73', 6), ('s74', 4), ('s75', 16), ('s76', 10), ('s77', 9), ('s78', 4), ('s79', 5), ('s80', 5), ('s81'

In [14]:
def generate_density_graph(graph_path, output_path=None):
    sortedList = None
    maxEdges = 0
    scaledEdges = 0
    heatmapbrain = None
    
    ## This finds the maximum number of edges and the densest node.
    G = nx.read_graphml(graph_path)

    maxEdges = 0
    densestNode = ""
#     for i in range(len(G.edges())):
#         if ((len(G.edges('s' + str(i))) > maxEdges)):
#             maxEdges = len(G.edges('s' + str(i)))
#             densestNode = "s" + str(i)
            
    for node in G.nodes():
        num_edges = len(G.edges(node))
        if (num_edges > maxEdges):
            maxEdges = num_edges
            densestNode = node

    ## Find each node with a given number of edges, from 0 edges to maxEdges

    ## Find and store number of edges for each node in storageDict
    ## Key is 's1', value is number of edges
    storageDict = {}

    for n in G.nodes_iter():
        storageDict[n] = len(G.edges(n))

    orderedNodesEdgeCounts = col.OrderedDict(sorted(storageDict.items(), key=lambda (key, value): int(key.split('s')[1])))

    ## Create ordered list to visualize data
    sortedList = sorted(storageDict.values())

    # Calculate basic statistics
    statisticsArray = np.array(sortedList)
    averageNumberEdges = np.mean(statisticsArray)
    stdNumberEdges = np.std(statisticsArray)

    print("average edge count:")
    print(averageNumberEdges)
    print("standard deviation edge count: ")
    print(stdNumberEdges)

    # using 95th percentile as upper limit (z = 1.96)
    upperLimit = averageNumberEdges + 1.96 * stdNumberEdges
    print("95th percentile: ")
    print(upperLimit)

    ##unused

    ## numberEdges is used for plotting (first element is edges for 's1', etc.)
    numberEdges = []
    k = 0
#     for i in range(1, (len(G.nodes()) + 1)):
#         numberEdges.append(orderedNodesEdgeCounts['s' + str(i)])
#         k = k + 1

    for node in G.nodes():
        numberEdges.append(orderedNodesEdgeCounts[node])

    ## Number of colors is maxEdges
    numColors = maxEdges;

    # scaledEdges = [float(numberEdges[i])/float(upperLimit) for i in range(len(numberEdges))]
    scaledEdges = [float(numberEdges[i])/float(maxEdges) for i in range(len(numberEdges))]

    ##Tweak this to change the heat map scaling for the points.  Remove outliers.

    ##Tweak this to change the heat map scaling for the points.  Remove outliers.
    ##False coloration heatmap below.  I've commented it out in this version b/c the rainbows
    ##are difficult to interpret.  I've included a newer red version.

    heatMapBrain = [
        # Let null values (0.0) have color rgb(0, 0, 0)
        [0, 'rgb(0, 0, 0)'],  # black

        [0.1, '#7f0000'],
        [0.2, '#7f0000'],

        [0.2, '#b30000'],
        [0.3, '#b30000'],

        [0.3, '#d7301f'],
        [0.4, '#d7301f'],

        [0.4, '#ef6548'],
        [0.5, '#ef6548'],

        [0.5, '#fc8d59'],
        [0.6, '#fc8d59'],

        [0.6, '#fdbb84'],
        [0.7, '#fdbb84'],

        [0.7, '#fdd49e'],
        [0.8, '#fdd49e'],

        [0.8, '#fee8c8'],
        [0.9, '#fee8c8'],

        [0.9, '#fff7ec'],
        [1.0, '#fff7ec']
    ]
    
    g = G

# def get_brain_figure(g, plot_title='') :
#     """
#     Returns the plotly figure object for vizualizing a 3d brain network.
#     g: networkX object of brain
#     """

    # grab the node positions from the graphML file
    V = nx.number_of_nodes(g)
    attributes = nx.get_node_attributes(g, 'coord')
    node_positions_3d = pd.DataFrame(columns=['x', 'y', 'z'], index=range(V))
    for n in g.nodes_iter():
        node_positions_3d.loc[n] = [int((re.findall('\d+', str(attributes[n])))[0]), int((re.findall('\d+', str(attributes[n])))[1]), int((re.findall('\d+', str(attributes[n])))[2])]

    # grab edge endpoints
    edge_x = []
    edge_y = []
    edge_z = []

    for e in g.edges_iter():
        source_pos = node_positions_3d.loc[e[0]]
        target_pos = node_positions_3d.loc[e[1]]

        edge_x += [source_pos['x'], target_pos['x'], None]
        edge_y += [source_pos['y'], target_pos['y'], None]
        edge_z += [source_pos['z'], target_pos['z'], None]

    Xlist = []
    for i in range(1, len(g.nodes()) + 1):
        Xlist.append(int((re.findall('\d+', str(attributes['s' + str(i)])))[0]))

    Ylist = []
    for i in range(1, len(g.nodes()) + 1):
        Ylist.append(int((re.findall('\d+', str(attributes['s' + str(i)])))[1]))

    Zlist = []
    for i in range(1, len(g.nodes()) + 1):
        Zlist.append(int((re.findall('\d+', str(attributes['s' + str(i)])))[2]))

    # Set tupleResolution to resolution input parameter
#     tupleResolution = resolution;

    # EG: for Aut1367, the spacing is (0.01872, 0.01872, 0.005).
#     xResolution = tupleResolution[0]
#     yResolution = tupleResolution[1]
#     zResolution = tupleResolution[2]
    # Now, to get the mm image size, we can multiply all x, y, z
    # to get the proper mm size when plotting.


    # node style
    node_trace = Scatter3d(x=[x for x in Xlist],
                           y=[x for x in Ylist],
                           z=[x for x in Zlist],
                           mode='markers',
                           # name='regions',
                           marker=Marker(symbol='dot',
                                         size=6,
                                         opacity=0,
                                         color=scaledEdges,
                                         colorscale=heatMapBrain),     
                                       # text=[str(r) for r in range(V)],
                                       # text=atlas_data['nodes'],
                           hoverinfo='text')
    # edge style
    '''edge_trace = Scatter3d(x=edge_x,
                           y=edge_y,
                           z=edge_z,
                           mode='lines',
                           line=Line(color='cyan', width=1),
                           hoverinfo='none')'''

    # axis style
    axis = dict(showbackground=False,
                showline=False,
                zeroline=False,
                showgrid=False,
                showticklabels=False)

    # overall layout
    layout = Layout(title=plot_title,
                    width=800,
                    height=900,
                    showlegend=False,
                    scene=Scene(xaxis=XAxis(axis),
                                yaxis=YAxis(axis),
                                zaxis=ZAxis(axis)),
                    margin=Margin(t=50),
                    hovermode='closest',
                    paper_bgcolor='rgba(1,1,1,1)',
                    plot_bgcolor='rgb(1,1,1)')

    data = Data([node_trace])
    fig = Figure(data=data, layout=layout)

#     return fig

# def generate_heat_map():
    # Get list of all possible number of edges, in order
    setOfAllPossibleNumEdges = set(sortedList)
    listOfAllPossibleNumEdges = list(setOfAllPossibleNumEdges)
    #listOfAllScaledEdgeValues = [listOfAllPossibleNumEdges[i]/upperLimit for i in range(len(listOfAllPossibleNumEdges))]
    listOfAllScaledEdgeValues = [listOfAllPossibleNumEdges[i]/float(maxEdges) for i in range(len(listOfAllPossibleNumEdges))]


    #heatMapBrain

    data = Data([
        Scatter(
            y=listOfAllPossibleNumEdges,
            marker=Marker(
                size=16,
                color=listOfAllPossibleNumEdges,
                colorbar=ColorBar(
                    title='Colorbar'
                ),
                colorscale=heatMapBrain,
            ),
            mode='markers')
    ])

    layout = Layout(title='false coloration scheme',
                        width=800,
                        height=900,
                        showlegend=False,
                        margin=Margin(t=50),
                        hovermode='closest',
                        xaxis=dict(
                            title='Number of Unique Colors',
                            titlefont=dict(
                            family='Courier New, monospace',
                            size=18,
                            color='#000000')
                            ),
                        yaxis=dict(
                            title='Number of Edges',
                            titlefont=dict(
                            family='Courier New, monospace',
                            size=18,
                            color='#000000')
                            ),
                        paper_bgcolor='rgba(255,255,255,255)',
                        plot_bgcolor='rgb(255,255,255)')

    mapping = Figure(data=data, layout=layout)
    #iplot(mapping, validate=False)
    
    if output_path != None:
        plotly.offline.plot(fig, filename=output_path)
        plotly.offline.plot(mapping, filename=output_path.split('.')[0] + '_heatmap.html')

    #plotly.offline.plot(mapping, filename = self._token + '/' + self._token + 'heatmap' + '.html')
    return fig, mapping

In [17]:
generate_density_graph('Fear199_2.graphml', output_path='Fear199_density.html')

IndexError: list index out of range

# Region stuff

In [29]:
test = nx.read_graphml('Fear199.graphml', node_type=str)
print(test.nodes())

['[110, 57, 308]', '[301, 157, 231]', '[124, 199, 334]', '[306, 160, 290]', '[125, 84, 401]', '[261, 209, 383]', '[201, 99, 128]', '[164, 173, 347]', '[192, 261, 446]', '[272, 112, 70]', '[170, 85, 360]', '[139, 176, 515]', '[362, 219, 307]', '[184, 154, 318]', '[403, 92, 360]', '[303, 216, 399]', '[163, 276, 402]', '[306, 230, 419]', '[215, 249, 264]', '[420, 152, 288]', '[65, 217, 262]', '[121, 203, 191]', '[303, 231, 199]', '[219, 203, 63]', '[358, 207, 221]', '[221, 222, 325]', '[325, 261, 301]', '[110, 74, 178]', '[136, 167, 446]', '[132, 132, 480]', '[199, 158, 160]', '[144, 160, 56]', '[142, 248, 225]', '[107, 195, 198]', '[312, 135, 362]', '[192, 83, 230]', '[290, 230, 363]', '[344, 128, 262]', '[269, 150, 375]', '[373, 181, 208]', '[378, 214, 201]', '[84, 115, 158]', '[248, 153, 425]', '[287, 223, 238]', '[254, 108, 295]', '[300, 115, 370]', '[207, 84, 120]', '[295, 25, 289]', '[313, 183, 475]', '[183, 123, 126]', '[160, 75, 290]', '[315, 39, 245]', '[332, 65, 287]', '[226, 16

In [32]:
s = ast.literal_eval(test.nodes()[0])
print(s[0])

110


In [36]:
points_path = 'Fear199_regions.csv'
thedata = np.genfromtxt(points_path, delimiter=',')
print(thedata)

[[  36.  162.  267.   48.  755.]
 [  37.  137.  315.   47.  755.]
 [  38.  115.  303.   43.  251.]
 ..., 
 [ 424.  124.  333.   43.  251.]
 [ 424.  162.  339.   40.  836.]
 [ 429.  126.  327.   41.  959.]]


In [48]:
points_path = 'Fear199_regions.csv'
thedata = np.genfromtxt(points_path, delimiter=',')

thedata = np.delete(thedata, [3], axis=1)
print(thedata)

[[  36.  162.  267.  755.]
 [  37.  137.  315.  755.]
 [  38.  115.  303.  251.]
 ..., 
 [ 424.  124.  333.  251.]
 [ 424.  162.  339.  836.]
 [ 429.  126.  327.  959.]]


In [60]:
a = np.array([1, 1, 1, 2, 3, 4, 4, 5])
au = np.unique(a)

print au

[1 2 3 4 5]


In [71]:
def generate_region_graph(token, points_path, output_path=None):
    font = {'weight' : 'bold',
        'size'   : 18}

    matplotlib.rc('font', **font)
    
#     points_path = 'Fear199_regions.csv'
    thedata = np.genfromtxt(points_path, dtype=int, delimiter=',')
    # deleting the brightness column
    thedata = np.delete(thedata, [3], axis=1)
    
    # Set tupleResolution to resolution input parameter
#     tupleResolution = resolution;

    # EG: for Aut1367, the spacing is (0.01872, 0.01872, 0.005).
#     xResolution = tupleResolution[0]
#     yResolution = tupleResolution[1]
#     zResolution = tupleResolution[2]
    # Now, to get the mm image size, we can multiply all x, y, z
    # to get the proper mm size when plotting.

    """Load the CSV of the ARA with CCF v2 (see here for docs:)"""
    ccf_txt = 'natureCCFOhedited.csv'

    ccf = {}
    with open(ccf_txt, 'rU') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            # row[0] is ccf atlas index, row[4] is string of full name
            ccf[row[0]] = row[4];
            # print row[0]
            # print row[4]
            # print ', '.join(row)

    """Save counts for each region into a separate CSV"""
    unique = [];

    for l in thedata:
        unique.append(l[3])

    uniqueNP = np.asarray(unique)
    allUnique = np.unique(uniqueNP)
    numRegionsA = len(allUnique)

    """
           First we download annotation ontology from Allen Brain Atlas API.
           It returns a JSON tree in which larger parent structures are divided into smaller children regions.
           For example the "corpus callosum" parent is has children "corpus callosum, anterior forceps", "genu of corpus callosum", "corpus callosum, body", etc
           """

    url = "http://api.brain-map.org/api/v2/structure_graph_download/1.json"
    jsonRaw = requests.get(url).content
    jsonDict = json.loads(jsonRaw)

    """
    Next we collect the names and ids of all of the regions.
    Since our json data is a tree we can walk through it in arecursive manner.
    Thus starting from the root...
    """
    root = jsonDict['msg'][0]
    """
    ...we define a recursive function ...
    """

    leafList = []

    def getChildrenNames(parent, childrenNames={}):
        if len(parent['children']) == 0:
            leafList.append(parent['id'])

        for childIndex in range(len(parent['children'])):
            child = parent['children'][childIndex]
            childrenNames[child['id']] = child['name']

            childrenNames = getChildrenNames(child, childrenNames)
        return childrenNames

    """
    ... and collect all of the region names in a dictionary with the "id" field as keys.
    """

    regionDict = getChildrenNames(root)

    ## Store most specific data
    specificRegions = [];

    for l in thedata:
        if l[3] in leafList:
            specificRegions.append(l)

    # Find all unique regions of brightest points (new)
    uniqueFromSpecific = [];

    for l in specificRegions:
        uniqueFromSpecific.append(l[3])

    uniqueSpecificNP = np.asarray(uniqueFromSpecific)
    allUniqueSpecific = np.unique(uniqueSpecificNP)
    numRegionsASpecific = len(allUniqueSpecific)

#     print "All unique specific region ID's:"
#     print allUniqueSpecific
    print "Total number of unique ID's:"
    print numRegionsASpecific  ## number of regions

    # Store and count the bright regions in each unique region (new)
    dictNumElementsRegionSpecific = {}
    num_points_by_region_dict = {}

    for i in range(numRegionsASpecific):
        counter = 0;
        for l in specificRegions:
            if l[3] == allUniqueSpecific[i]:
                counter = counter + 1;
                dictNumElementsRegionSpecific[ccf[str(l[3])]] = counter;
                num_points_by_region_dict[l[3]] = counter

    region_names = dictNumElementsRegionSpecific.keys()
    number_repetitions = dictNumElementsRegionSpecific.values()

    from itertools import izip

    with open(token + 'specific_counts.csv', 'wb') as write:
        writer = csv.writer(write)
        writer.writerows(izip(region_names, number_repetitions))

    specificRegionsNP = np.asarray(specificRegions)

    region_dict = OrderedDict()

    for l in specificRegionsNP:
        trace = ccf[str(l[3])]
        # trace = 'trace' + str(l[3])
        if trace not in region_dict:
            region_dict[trace] = np.array([[l[0], l[1], l[2], l[3]]])
            # print 'yay'
        else:
            tmp = np.array([[l[0], l[1], l[2], l[3]]])
            region_dict[trace] = np.concatenate((region_dict.get(trace, np.zeros((1, 4))), tmp), axis=0)
            # print 'nay'

    current_palette = sns.color_palette("husl", numRegionsA)
    # print current_palette

    data = []
    for i, key in enumerate(region_dict):
        trace = region_dict[key]
        tmp_col = current_palette[i]
        tmp_col_lit = 'rgb' + str(tmp_col)
        temp = str(np.unique(trace[:, 3])).replace("[", "")
        final = temp.replace("]", "")

        trace_scatter = Scatter3d(
            x=[x for x in trace[:, 0]],
            y=[x for x in trace[:, 1]],
            z=[x for x in trace[:, 2]],

            mode='markers',
            name=ccf[final],
            marker=dict(
                size=1.2,
                color=tmp_col_lit,  # 'purple',                # set color to an array/list of desired values
                colorscale='Viridis',  # choose a colorscale
                opacity=0.2
            )
        )

        data.append(trace_scatter)


    layout = Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        paper_bgcolor='rgb(0,0,0)',
        plot_bgcolor='rgb(0,0,0)'
    )

    fig = Figure(data=data, layout=layout)
    
    if output_path != None:
        plotly.offline.plot(fig, filename=output_path)
    
    return fig, num_points_by_region_dict


In [67]:
figure = generate_region_graph('Fear199', 'Fear199_regions.csv', 'Fear199_region.html')

Total number of unique ID's:
556


In [72]:
f, d = generate_region_graph('Fear199', 'Fear199_regions.csv')

Total number of unique ID's:
556


In [73]:
print d

{2: 2, 7: 12, 9: 30, 12: 1, 15: 3, 17: 19, 19: 4, 20: 2, 23: 15, 26: 46, 27: 1, 28: 33, 33: 13, 36: 1, 41: 4, 42: 2, 45: 1, 50: 1, 52: 25, 53: 1, 55: 2, 56: 144, 59: 8, 60: 3, 61: 2, 64: 6, 68: 10, 69: 13, 74: 5, 77: 1, 82: 4, 83: 5, 84: 7, 87: 3, 91: 8, 92: 17, 96: 7, 99: 9, 101: 12, 10710: 15, 105: 1, 106: 1, 113: 26, 114: 4, 115: 1, 118: 3, 120: 3, 121: 2, 122: 22, 123: 5, 130: 7, 131: 19, 132: 1, 133: 2, 136: 56, 137: 7, 139: 31, 10723: 4, 146: 96, 148: 4, 149: 19, 155: 29, 156: 12, 160: 22, 162: 3, 163: 7, 167: 9, 169: 7, 171: 13, 175: 3, 180: 20, 181: 23, 183: 24, 186: 4, 187: 11, 188: 5, 189: 9, 191: 15, 192: 3, 194: 58, 195: 6, 196: 3, 197: 1, 199: 20, 200: 2, 201: 75, 202: 29, 203: 2, 204: 7, 206: 3, 209: 16, 211: 17, 212: 73, 214: 11, 215: 25, 216: 7, 217: 10, 218: 33, 220: 101, 222: 1, 224: 9, 225: 13, 226: 28, 228: 20, 231: 1, 232: 11, 233: 17, 234: 11, 236: 16, 238: 3, 240: 2, 241: 20, 243: 7, 244: 88, 246: 6, 248: 2, 250: 14, 251: 11, 252: 18, 255: 11, 256: 11, 257: 2, 25